In [ ]:
#| default_exp mstl

# MFLES model

In [1]:
#| hide
#from nbdev.showdoc import add_docs, show_doc

In [2]:
#| export
from typing import Dict, List, Optional, Union

import numpy as np
import pandas as pd
from numba import jit, njit, vectorize
import itertools

ModuleNotFoundError: No module named 'pandas'

In [ ]:
#utility functions

def cap_outliers(series, outlier_cap=3):
    mean = np.mean(series)
    std = np.std(series)
    series = series.clip(min=mean - outlier_cap * std,
                         max=mean + outlier_cap * std)
    return series

@njit
def calc_mse(actual, fitted):
    mse = np.zeros(len(actual))
    for i, val in enumerate(zip(actual, fitted)):
        mse[i] = (val[0] - val[1])**2
    return np.mean(mse)

@njit
def set_fourier(period):
    if period < 10:
        fourier = 5
    elif period < 70:
        fourier = 10
    else:
        fourier = 15
    return fourier

@njit
def calc_trend_strength(resids, deseasonalized):
    return max(0, 1-(np.var(resids)/np.var(deseasonalized)))

@njit
def calc_seas_strength(resids, detrended):
    return max(0, 1-(np.var(resids)/np.var(detrended)))

@njit
def calc_rsq(y, fitted):
    try:
        mean_y = np.mean(y)
        ssres = 0
        sstot = 0
        for vals in zip(y, fitted):
            sstot += (vals[0] - mean_y) ** 2
            ssres += (vals[0] - vals[1]) ** 2
        return 1 - (ssres / sstot)
    except:
        return 0

@njit
def calc_cov(y, mult=1):
    if mult:
        # source http://medcraveonline.com/MOJPB/MOJPB-06-00200.pdf
        return np.sqrt(np.exp(np.log(10)*(np.std(y)**2) - 1))
    else:
        return np.std(y) / np.mean(y)

@njit
def fastsin(x):
    return np.sin(2* np.pi * x)

@njit
def fastcos(x):
    return np.cos(2 * np.pi * x)

@njit
def get_seasonality_weights(y, seasonal_period):
    seasonal_sample_weights = []
    weight = 0
    for i in range(len(y)):
        if (i) % seasonal_period == 0:
            weight += 1
        seasonal_sample_weights.append(weight)
    return seasonal_sample_weights

In [ ]:
#feature engineering functions
@njit
def fourier(y, seasonal_period, fourier_order):
    n = len(y)
    m = fourier_order * 2
    results = np.zeros((n, m))
    for j in range(fourier_order):
        for i in range(n):
            x = (j + 1) * i
            results[i, j] = fastcos(x) / seasonal_period
            results[i, m-j-1] = fastsin(x) / seasonal_period
    return results

@njit
def get_fourier_series(length, seasonal_period, fourier_order):
    x = 2 * np.pi * np.arange(1, fourier_order + 1) / seasonal_period
    t = np.arange(1, length + 1).reshape(-1, 1)
    x = x * t
    fourier_series = np.concatenate((np.cos(x), np.sin(x)), axis=1)
    return fourier_series

@njit
def get_basis(y, n_changepoints, decay=-1, gradient_strategy=0):
    y = y.copy()
    y -= y[0]
    n = len(y)
    if gradient_strategy:
        gradients = np.abs(y[:-1] - y[1:])
    initial_point = y[0]
    final_point = y[-1]
    mean_y = np.mean(y)
    changepoints = np.zeros(shape=(len(y), n_changepoints + 1))
    array_splits = []
    for i in range(1, n_changepoints + 1):
        i = n_changepoints - i + 1
        if gradient_strategy:
            cps = np.argsort(gradients)[::-1]
            cps = cps[cps > .1 * len(gradients)]
            cps = cps[cps < .9 * len(gradients)]
            split_point = cps[i-1]
            array_splits.append(y[:split_point])
        else:
            split_point = len(y)//i
            array_splits.append(y[:split_point])
            y = y[split_point:]
    len_splits = 0
    for i in range(n_changepoints):
        if gradient_strategy:
            len_splits = len(array_splits[i])
        else:
            len_splits += len(array_splits[i])
        moving_point = array_splits[i][-1]
        left_basis = np.linspace(initial_point,
                                  moving_point,
                                  len_splits)
        if decay is None:
            end_point = final_point
        else:
            if decay == -1:
                dd = moving_point**2 / (mean_y**2)
                if dd > .99:
                    dd = .99
                if dd < .001:
                    dd = .001
                end_point = moving_point - ((moving_point - final_point) * (1 - dd))
            else:
                end_point = moving_point - ((moving_point - final_point) * (1 - decay))
        right_basis = np.linspace(moving_point,
                                  end_point,
                                  n - len_splits + 1)
        changepoints[:, i] = np.append(left_basis, right_basis[1:])
    changepoints[:, i+1] = np.ones(n)
    return changepoints

@jit
def get_future_basis(basis_functions, forecast_horizon):
    n_components = np.shape(basis_functions)[1]
    slopes = np.gradient(basis_functions)[0][-1, :]
    future_basis = np.array(np.arange(0, forecast_horizon + 1))
    future_basis += len(basis_functions)
    future_basis = np.transpose([future_basis] * n_components)
    future_basis = future_basis * slopes
    future_basis = future_basis + (basis_functions[-1, :] - future_basis[0, :])
    return future_basis[1:, :]

In [ ]:
#different models
@njit
def fsign(f):
    if f == 0:
        return 0
    elif f > 0:
        return 1.0
    else:
        return -1.0

@njit
def soft_threshold(rho, alpha):
    """Soft threshold function used for Lasso regression"""
    if rho < -alpha:
        return rho + alpha
    elif rho > alpha:
        return rho - alpha
    else:
        return 0

@jit
def lasso_nb(X, y, alpha, tol=0.001, maxiter=10000):
    n, p = X.shape
    beta = np.zeros(p)
    R = y.copy()
    norm_cols_X = (X ** 2).sum(axis=0)
    resids = []
    prev_cost = 10e10
    for n_iter in range(maxiter):
        for ii in range(p):
            beta_ii = beta[ii]
            if beta_ii != 0.:
                R += X[:, ii] * beta_ii
            tmp = np.dot(X[:, ii], R)
            beta[ii] = fsign(tmp) * max(abs(tmp) - alpha, 0) / (.00001 + norm_cols_X[ii])
            if beta[ii] != 0.:
                R -= X[:, ii] * beta[ii]
        cost = (np.sum((y - X @ beta)**2) + alpha * np.sum(np.abs(beta))) / n
        resids.append(cost)
        if prev_cost - cost < tol:
            break
        else:
            prev_cost = cost
    return beta

@vectorize
def calc_slope(x1,y1,x2,y2):
    xd = x2-x1
    if xd == 0:
        slope = 0
    else:
        slope = (y2-y1) / (xd)
    return slope

@njit
def siegel_repeated_medians(x,y):
    # Siegel repeated medians regression
    n_total = x.size
    slopes = np.empty((n_total), dtype=y.dtype)
    ints = np.empty((n_total), dtype=y.dtype)
    slopes_sub = np.empty((n_total-1), dtype=y.dtype)
    for i in range(n_total):
        for j in range(n_total):            
            if i == j:
                continue
            slopes_sub[j] = calc_slope(x[i],y[i],x[j],y[j])
        slopes[i] = np.median(slopes_sub)
        ints[i] = y[i] - slopes[i]*x[i]
    trend = x * np.median(slopes) + np.median(ints)
    return trend

@njit
def ses(y, alpha):
    results = np.zeros(len(y))
    results[0] = y[0]
    for i in range(1, len(y)):
        results[i] = alpha * y[i] + (1 - alpha) * results[i - 1]
    return results

@njit
def ses_ensemble(y, min_alpha=.05, max_alpha=1, smooth=0, order=1):
    #bad name but does either a ses ensemble or simple moving average
    results = np.zeros(len(y))
    iters = np.arange(min_alpha, max_alpha, .05)
    if smooth:
        for alpha in iters:
            results += ses(y, alpha)
        results = results / len(iters)
    else:
        results[:order] = y[:order]
        for i in range(1 + order, len(y)):
            results[i] += np.sum(y[i-order:i+1])/(order+1)
        results[:order + 1] = y[:order + 1] #fix this
    return results

@njit
def fast_ols(x, y):
    """Simple OLS for two data sets."""
    M = x.size

    x_sum = 0.
    y_sum = 0.
    x_sq_sum = 0.
    x_y_sum = 0.

    for i in range(M):
        x_sum += x[i]
        y_sum += y[i]
        x_sq_sum += x[i] ** 2
        x_y_sum += x[i] * y[i]

    slope = (M * x_y_sum - x_sum * y_sum) / (M * x_sq_sum - x_sum**2)
    intercept = (y_sum - slope * x_sum) / M

    return slope * x + intercept

@jit
def median(y, seasonal_period):
    n = len(y)
    if seasonal_period is None:
        return np.median(y) * np.ones(n)
    else:
        medians = np.zeros(n)
        for i in range(int(n / seasonal_period)):
            left = i * seasonal_period
            right = (1 + i) * seasonal_period
            medians[left: right] = np.median(y[left: right])
        remainder = n % seasonal_period
        if remainder:
            medians[right:] = np.median(y[left + remainder: ])
        return medians

@jit
def ols(X, y):
    coefs = np.linalg.pinv(X.T.dot(X)).dot(X.T.dot(y))
    return  np.sum(coefs * X, axis=1)

@jit
def wls(X, y, weights):
    weighted_X_T = X.T @ np.diag(weights)
    coefs = np.linalg.pinv(weighted_X_T.dot(X)).dot(weighted_X_T.dot(y))
    return  np.sum(coefs * X, axis=1)

@jit
def _ols(X, y):
    coefs = np.linalg.pinv(X.T.dot(X)).dot(X.T.dot(y))
    return  coefs

@jit
def ridge(X, y, lam):
    eye = np.eye(X.shape[1])
    coefs = np.linalg.pinv(X.T.dot(X) + eye@lam).dot(X.T.dot(y))
    return  np.sum(coefs * X, axis=1)

class OLS:
    def __init__(self):
        pass
    def fit(self, X, y):
        self.coefs = _ols(X, y)
    def predict(self, X):
        return  np.sum(self.coefs * X, axis=1)

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
from itertools import cycle
from tqdm import tqdm
import json

class MFLES:
    def __init__(self, verbose=1, robust=None):
        self.penalty = None
        self.trend = None
        self.seasonality = None
        self.robust = robust
        self.const = None
        self.aic = None
        self.upper = None
        self.lower= None
        self.exogenous_models = None
        self.verbose = verbose
        self.predicted = None

    def fit(self,
            y,
            seasonal_period=None,
            X=None,
            fourier_order=None,
            ma=None,
            alpha=.1,
            decay=-1,
            n_changepoints=.25,
            seasonal_lr=.9,
            rs_lr=1,
            exogenous_lr=1,
            exogenous_estimator=OLS,
            exogenous_params={},
            linear_lr=.9,
            cov_threshold=.7,
            moving_medians=False,
            max_rounds=50,
            min_alpha=.05,
            max_alpha=1.0,
            round_penalty=0.0001,
            trend_penalty=True,
            multiplicative=None,
            changepoints=True,
            smoother=False,
            seasonality_weights=False):
        """
        

        Parameters
        ----------
        y : TYPE
            DESCRIPTION.
        seasonal_period : TYPE, optional
            DESCRIPTION. The default is None.
        fourier_order : TYPE, optional
            DESCRIPTION. The default is None.
        ma : TYPE, optional
            DESCRIPTION. The default is None.
        alpha : TYPE, optional
            DESCRIPTION. The default is .1.
        decay : TYPE, optional
            DESCRIPTION. The default is -1.
        n_changepoints : TYPE, optional
            DESCRIPTION. The default is .25.
        seasonal_lr : TYPE, optional
            DESCRIPTION. The default is .9.
        rs_lr : TYPE, optional
            DESCRIPTION. The default is 1.
        linear_lr : TYPE, optional
            DESCRIPTION. The default is .9.
        cov_threshold : TYPE, optional
            DESCRIPTION. The default is .7.
        moving_medians : TYPE, optional
            DESCRIPTION. The default is False.
        max_rounds : TYPE, optional
            DESCRIPTION. The default is 10.
        min_alpha : TYPE, optional
            DESCRIPTION. The default is .05.
        max_alpha : TYPE, optional
            DESCRIPTION. The default is 1.0.
        trend_penalty : TYPE, optional
            DESCRIPTION. The default is True.
        multiplicative : TYPE, optional
            DESCRIPTION. The default is None.
        changepoints : TYPE, optional
            DESCRIPTION. The default is True.
        smoother : TYPE, optional
            DESCRIPTION. The default is False.

        Returns
        -------
        None.

        """
        if cov_threshold == -1:
            cov_threshold = 10000
        n = len(y)
        if n < 4 or np.all(y == np.mean(y)):
            if self.verbose:
                if n < 4:
                    print('series is too short (<4), defaulting to naive')
                else:
                    print(f'input is constant with value {y[0]}, defaulting to naive')
            self.trend = np.append(y[-1], y[-1])
            self.seasonality = np.zeros(len(y))
            self.trend_penalty = False
            self.mean = 0
            self.std = 0
            return np.tile(y[-1], len(y))
        og_y = y
        self.og_y = og_y
        y = y.copy()
        if n_changepoints is None:
            changepoints = False
        if isinstance(n_changepoints, float) and n_changepoints < 1:
            n_changepoints = int(n_changepoints * n)
        self.linear_component = np.zeros(n)
        self.seasonal_component = np.zeros(n)
        self.ses_component = np.zeros(n)
        self.median_component = np.zeros(n)
        self.exogenous_component = np.zeros(n)
        self.exogenous_lr = exogenous_lr
        self.exo_model = []
        self.round_cost = []
        if multiplicative is None:
            if seasonal_period is None:
                multiplicative = False
            else:
                multiplicative = True
            if min(y) <= 0:
                multiplicative = False
        if multiplicative:
            self.const = y.min()
            y = np.log(y)
        else:
            self.const = None
            self.std = np.std(y)
            self.mean = np.mean(y)
            y = (y - self.mean) / self.std
        if seasonal_period is not None:
            if not isinstance(seasonal_period, list):
                seasonal_period = [seasonal_period]
        self.trend_penalty = trend_penalty
        if moving_medians and seasonal_period is not None:
            fitted = median(y, max(seasonal_period))
        else:
            fitted = median(y, None)
        self.median_component += fitted
        self.trend = np.append(fitted.copy()[-1:], fitted.copy()[-1:])
        mse = None
        equal = 0
        if ma is None:
            ma_cycle = cycle([1])
        else:
            if not isinstance(ma, list):
                ma = [ma]
            ma_cycle = cycle(ma)
        if seasonal_period is not None:
            seasons_cycle = cycle(list(range(len(seasonal_period))))
            self.seasonality = np.zeros(max(seasonal_period))
            fourier_series = []
            for period in seasonal_period:
                if fourier_order is None:
                    fourier = set_fourier(period)
                else:
                    fourier = fourier_order
                fourier_series.append(get_fourier_series(n,
                                                    period,
                                                    fourier))
            if seasonality_weights:
                cycle_weights = []
                for period in seasonal_period:
                    cycle_weights.append(get_seasonality_weights(y, period))
        else:
            self.seasonality = None
        for i in range(max_rounds):
            resids = y - fitted
            if mse is None:
                mse = calc_mse(y, fitted)
            else:
                if mse <= calc_mse(y, fitted):
                    if equal == 6:
                        break
                    equal += 1
                else:
                    mse = calc_mse(y, fitted)
                self.round_cost.append(mse)
            if seasonal_period is not None:
                seasonal_period_cycle = next(seasons_cycle)
                if seasonality_weights:
                    seas = wls(fourier_series[seasonal_period_cycle],
                               resids,
                               cycle_weights[seasonal_period_cycle])
                else:
                    seas = ols(fourier_series[seasonal_period_cycle],
                               resids)
                seas = seas * seasonal_lr
                component_mse = calc_mse(y, fitted + seas)
                if mse > component_mse:
                    mse = component_mse
                    fitted += seas
                    resids = y - fitted
                    self.seasonality += np.resize(seas[-seasonal_period[seasonal_period_cycle]:],
                                                  len(self.seasonality))
                    self.seasonal_component += seas
            if X is not None and i > 0:
                model_obj = exogenous_estimator(**exogenous_params)
                model_obj.fit(X, resids)
                self.exo_model.append(model_obj)
                _fitted_values = model_obj.predict(X) * exogenous_lr
                self.exogenous_component += _fitted_values
                fitted += _fitted_values
                resids = y - fitted
            if i % 2: #if even get linear piece, allows for multiple seasonality fitting a bit more
                if self.robust:
                    tren = siegel_repeated_medians(x=np.arange(n),
                                    y=resids)
                else:
                    if i==1 or not changepoints:
                        tren = fast_ols(x=np.arange(n),
                                        y=resids)
                    else:
                        cps = min(n_changepoints, int(.1*n))
                        lbf = get_basis(y=resids,
                                        n_changepoints=cps,
                                        decay=decay)
                        tren = np.dot(lbf, lasso_nb(lbf, resids, alpha=alpha))
                        tren = tren * linear_lr
                component_mse = calc_mse(y, fitted + tren)
                if mse > component_mse:
                    mse = component_mse
                    fitted += tren
                    self.linear_component += tren
                    self.trend += tren[-2:]
                    if i == 1:
                        self.penalty = calc_rsq(resids, tren)
            elif i > 4 and not i % 2:
                if smoother is None:
                    if seasonal_period is not None:
                        len_check = int(max(seasonal_period))
                    else:
                        len_check = 12
                    if resids[-1] > np.mean(resids[-len_check:-1]) + 3 * np.std(resids[-len_check:-1]):
                        smoother = 0
                    if resids[-1] < np.mean(resids[-len_check:-1]) - 3 * np.std(resids[-len_check:-1]):
                        smoother = 0
                    if resids[-2] > np.mean(resids[-len_check:-2]) + 3 * np.std(resids[-len_check:-2]):
                        smoother = 0
                    if resids[-2] < np.mean(resids[-len_check:-2]) - 3 * np.std(resids[-len_check:-2]):
                        smoother = 0
                    if smoother is None:
                        smoother = 1
                    else:
                        resids[-2:] = cap_outliers(resids, 3)[-2:]
                tren = ses_ensemble(resids,
                                    min_alpha=min_alpha,
                                    max_alpha=max_alpha,
                                    smooth=smoother*1,
                                    order=next(ma_cycle)
                                    )
                tren = tren * rs_lr
                component_mse = calc_mse(y, fitted + tren)
                if mse > component_mse + round_penalty * mse:
                    mse = component_mse
                    fitted += tren
                    self.ses_component += tren
                    self.trend += tren[-1]
            if i == 0: #get deasonalized cov for some heuristic logic
                if self.robust is None:
                    try:
                        if calc_cov(resids, multiplicative) > cov_threshold:
                            self.robust = True
                        else:
                            self.robust = False
                    except:
                        self.robust = True

            if i == 1:
                resids = cap_outliers(resids, 5) #cap extreme outliers after initial rounds
        if multiplicative:
            fitted = np.exp(fitted)
        else:
            fitted = self.mean + (fitted * self.std)
        self.multiplicative = multiplicative
        self.fitted = fitted
        return fitted

    def predict(self, forecast_horizon, X=None):
        last_point = self.trend[1]
        slope = last_point - self.trend[0]
        if self.trend_penalty and self.penalty is not None:
            slope = slope * max(0, self.penalty)
        self.predicted_trend = slope * np.arange(1, forecast_horizon + 1) + last_point
        if self.seasonality is not None:
            predicted = self.predicted_trend + np.resize(self.seasonality, forecast_horizon)
        else:
            predicted = self.predicted_trend
        if X is not None:
            for model in self.exo_model:
                predicted += model.predict(X) * self.exogenous_lr
        if self.const is not None:
            predicted = np.exp(predicted)
        else:
            predicted = self.mean + (predicted * self.std)
        self.predicted = predicted
        return predicted



In [ ]:
#| hide
url = "https://raw.githubusercontent.com/tidyverts/tsibbledata/master/data-raw/vic_elec/VIC2015/demand.csv"
df = pd.read_csv(url)
df["Date"] = df["Date"].apply(
    lambda x: pd.Timestamp("1899-12-30") + pd.Timedelta(x, unit="days")
)
df["ds"] = df["Date"] + pd.to_timedelta((df["Period"] - 1) * 30, unit="m")
timeseries = df[["ds", "OperationalLessIndustrial"]]
timeseries.columns = [
    "ds",
    "y",
]  # Rename to OperationalLessIndustrial to y for simplicity.

# Filter for first 149 days of 2012.
start_date = pd.to_datetime("2012-01-01")
end_date = start_date + pd.Timedelta("149D")
mask = (timeseries["ds"] >= start_date) & (timeseries["ds"] < end_date)
timeseries = timeseries[mask]

# Resample to hourly
timeseries = timeseries.set_index("ds").resample("H").sum()
timeseries.head()

# decomposition
mfles = MFLES()
fitted = mfles.fit(y=timeseries.y.values, seasonal_period=[24, 24 * 7])
predicted = mfles.predict(forecast_horizon=24)